In [1]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
from six.moves import xrange

## Read In

In [2]:
def load(vocab_size):
    ppp = nltk.data.load('../../Downloads/ppp.txt', encoding='utf8')
    words_p = nltk.tokenize.wordpunct_tokenize(ppp)[130:]
    alw = nltk.data.load('../../Downloads/alw.txt', encoding='utf8')
    words_a = nltk.tokenize.wordpunct_tokenize(alw)[143:]
    words = words_a + words_p
    KB = nx.Graph()
    KB.add_nodes_from(np.arange(vocab_size))
    #return get_sentences(words), KB
    return words, KB

## Turn into list of Sentences

In [3]:
def get_sentences(words):
    sentences_word  = []
    sent_word = []
    for index, word in enumerate(words):
        if word in ['?','.','!']:
            sent_word += [word]
            sentences_word += [sent_word]
            sent_word = []
        else:
            sent_vec += [comb_embed[dictionary[word]]]
            sent_word += [word]
    return sentences_word

## Generate Each Batch
-- a useful thing would be for batches to be made of few words, a couple sentences, and then be taken from there. idk.

In [4]:
data_index = 0
def gen_batch(data, batch_size, skip_window, num_skips):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels


## Turn Vocab into Indices
-- This was process, but I think this is useful regardless. I will need to change it to accomodate ..... stuff?

In [5]:
def build_vocab(words, dic, revdic, vocab_size):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocab_size - 1))
    dictionary = dict()
    for word, _ in count:
        dic[word] = len(dic)
    data = list()
    unk_count = 0
    for word in words:
        if word in dic:
            index = dic[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    revdic = dict(zip(dic.values(), dic.keys()))
    return data, dic, revdic

## Set Up Graph Building

Basically we want to make sure that things are being added to the graph. How do we do this. Well. I think we should take them as if they were read in line and then only add things like that. This way we can do all our learning right good without having to stop and add to the graph

Adding node likelihood : http://ieeexplore.ieee.org/document/7266560/

In [6]:
def update_graph(KB, data):
    data = np.array(data)
    KB.add_edges_from(np.stack([data[:-1], np.roll(data, -1)[:-1]], 1))
    return KB

In [8]:
num_read = 1000
def main():
    vocab_size = 5000
    words, KB = load(vocab_size)
    data, dic, revdic = [],{},{}
    
    batch_size = 128
    embedding_size = 100  # Dimension of the embedding vector.
    skip_window = 2      # How many words to consider left and right.
    num_skips = 4         # How many times to reuse an input to generate a label.
    
    # We pick a random validation set to sample nearest neighbors. Here we limit the
    # validation samples to the words that have a low numeric ID, which by
    # construction are also the most frequent.
    valid_size = 50    # Random set of words to evaluate similarity on.
    valid_window = 200  # Only pick dev samples in the head of the distribution.
    valid_examples = np.random.choice(valid_window, valid_size, replace=False)
    num_sampled = 70    # Number of negative examples to sample.
    num_steps = 10000
    
    num_batches = 1
    
    KB = 
#    embeddings = np.random.uniform(-1, 1, (vocab_size, embedding_size))
#    for i in range(num_batches):
#        data, dic, revdic = build_vocab(words, dic, revdic, vocab_size)
#        KB = update_graph(KB, data)
#         embeddings = W2V(batch_size, embedding_size, skip_window,
#                         num_skips, valid_size, valid_window,
#                         valid_examples, num_sampled, len(embeddings),
#                         num_steps, revdic, embeddings, data)
    return KB

KB = main()

In [37]:
def W2V(batch_size, embedding_size, skip_window, num_skips, valid_size,
       valid_window, valid_examples, num_sampled, vocabulary_size,
       num_steps, revdic, embeds, data):
    graph = tf.Graph()
    
    with graph.as_default():
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        
        with tf.device('/cpu:0'):
            
            embeddings = tf.Variable(tf.constant(embeds.astype(np.float32),
                                        shape=[vocabulary_size, embedding_size]))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size], 
                               stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
        loss = tf.reduce_mean(
            tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                          num_sampled, vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 
                                     1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
        similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)
        
        init = tf.initialize_all_variables()
    
    with tf.Session(graph=graph) as session:
        
        init.run()
        print("Initialized")
        #saver = tf.train.Saver({'In'})
        
        average_loss = 0
        for step in xrange(num_steps):
            batch_inputs, batch_labels = gen_batch(
                data, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs : batch_inputs, 
                         train_labels : batch_labels}
            
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val

        
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print("Average loss at step ", step, ": ", average_loss)
                average_loss = 0

            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 5000 == 0:
                sim = similarity.eval()
                for i in xrange(valid_size):
                    valid_word = revdic[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log_str = "Nearest to %s:" % valid_word
                    for k in xrange(top_k):
                        close_word = revdic[nearest[k]]
                        log_str = "%s %s," % (log_str, close_word)
                    print(log_str)

        final_embeddings = normalized_embeddings.eval()
            
        
    

In [ ]:
def TFKMC(vectors, n_clusters=1000, max_iter=100000):
    mbatch = MiniBatchKMeans(n_clusters=n_clusters, batch_size=len(vectors), max_iter=max_iter)
    centroids = mbatch.fit(vectors)

## What To Do

### 1. Read Gradient Descent Site
### 2. Read the Candidate Sampling Thing
### 3. Implement KMeans MiniBatch in TensorFlow
### 4. Write Graph Adding Code
### 5. Write Processing for Transitions into Graph
### 6. Implement Word2Vec Using Graphs
